In [ ]:
import pyspark

## Steps:

1. Create Azure Synapse Workspace with ADLS Gen2 Storage Account.

2. Connect the Azure Synapse Workspace ADLS Gen2 Storage Account to an App.

3. Create a Dedicated SQL Pool under the created Workspace.

4. Copy the JDBC SQL Authentication URL.

5. Set up ADLS Gen2 app Connection and Azure Synapse Analytics Connection in Notebook.

6. Mount the Storage Folder & read data to a DataFrame.

7. Establish the Blob Storage connection (without mount point); necessary for `tempDir`.

8. Read Write data to Dedicated SQL Pool.

#### Establish the connection to ADLS Gen2 and Azure Synapse Analytics

In [ ]:
# App (configured to ADLS Gen2 App) Details
client_id = ""
tenant_id = ""
client_secret = ""
    
# Defining the service principal credentials for the Azure storage account
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type",  "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint", "https://login.microsoftonline.com/{}/oauth2/token".format(tenant_id))
    
# Defining a separate set of service principal credentials for Azure Synapse Analytics (If not defined, the connector will use the Azure storage account credentials)
spark.conf.set("spark.databricks.sqldw.jdbc.service.principal.client.id", client_id)
spark.conf.set("spark.databricks.sqldw.jdbc.service.principal.client.secret", client_secret)

#### Mount the ADLS Gen2 Storage Account & read data to DataFrame

In [ ]:
# connect and mount the storage folder of ADLS Gen2 Storage Account
mount_point = "/mnt/Gen2"

try:
    dbutils.fs.mount(
        source = storage_endpoint,
        mount_point = mount_point,
        extra_configs = configs
    )
except Exception as e:
    print("Error: \n",e)
    
# display file path
display(dbutils.fs.ls("/mnt/Gen2/CustomerFiles/"))

path,name,size,modificationTime
dbfs:/mnt/Gen2/CustomerFiles/part-00000-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-108-1-c000.csv,part-00000-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-108-1-c000.csv,2423786,1672578617000
dbfs:/mnt/Gen2/CustomerFiles/part-00001-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-109-1-c000.csv,part-00001-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-109-1-c000.csv,2423268,1672578616000
dbfs:/mnt/Gen2/CustomerFiles/part-00002-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-110-1-c000.csv,part-00002-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-110-1-c000.csv,2427801,1672578618000
dbfs:/mnt/Gen2/CustomerFiles/part-00003-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-111-1-c000.csv,part-00003-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-111-1-c000.csv,2419248,1672578617000
dbfs:/mnt/Gen2/CustomerFiles/part-00004-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-112-1-c000.csv,part-00004-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-112-1-c000.csv,2425732,1672578617000
dbfs:/mnt/Gen2/CustomerFiles/part-00005-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-113-1-c000.csv,part-00005-tid-3200334632332214470-9b4dec79-7e2e-495d-8657-3b5457ed3753-113-1-c000.csv,2419015,1672578616000


In [ ]:
# Read the files from ADLS Gen2 Container
df1 = spark.read.format("csv").option("header", True).option("inferSchema", True).load("dbfs:/mnt/Gen2/CustomerFiles/part-0*.csv")

df1.count()

Out[54]: 90000

In [ ]:
df1.show(5)

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|C_CUSTKEY|            C_NAME|           C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
|    35165|Customer#000035165|    eNQSvDTld1 f7JmY|          0|10-173-541-5438|  4767.46|  AUTOMOBILE|special excuses. ...|
|    30597|Customer#000030597|          S9s1dDut8Q|          0|10-607-243-5581|  -639.62|   FURNITURE|lithely ruthless ...|
|    42279|Customer#000042279|    ABcVdNnA3JFB7bK5|          0|10-934-981-2863|  2236.39|   MACHINERY|the even deposits...|
|    42578|Customer#000042578|l6VNaE7iSZFtkSC5f...|          0|10-281-998-8028|   6429.8|    BUILDING|y alongside of th...|
|    37854|Customer#000037854|  dL6LCTLpY9hjLTrZ7g|          0|10-909-820-4270|  9549.78|    BUILDING|inder blithely de...|
+-------

#### Connect to Blob Storage

In [ ]:
# BLOB STORAGE IS NEEDED FOR tempDir
# blob storage details
blob_storage_account = "<blob_storage_account>" + ".blob.core.windows.net"
blob_container = ""
blob_access_key = ""

# configure blob storage connection
acntInfo = "fs.azure.account.key.{}".format(blob_storage_account)
spark.conf.set(
    acntInfo,
    blob_access_key
)

#### Read Write Operations

In [ ]:
sqlDwUrl = "<JDBC_SQL_Authenticate_URL>"
tempDir = "wasbs://{}@{}/<folder_name>".format(blob_container, blob_storage_account)

In [ ]:
# READ DATA FROM DEDICATED SQL POOL
df = spark.read \
   .format("com.databricks.spark.sqldw") \
   .option("url", sqlDwUrl) \
   .option("tempDir", tempDir) \
   .option("forwardSparkAzureStorageCredentials", "true") \
   .option("dbTable", "<table_name>") \
   .load()

In [ ]:
df.count()

Out[16]: 90000

In [ ]:
df.show()

In [ ]:
# Write Data To a new Table
df1.write\
    .format("com.databricks.spark.sqldw")\
    .option("url", sqlDwUrl)\
    .option("forwardSparkAzureStorageCredentials", "true")\
    .option("dbTable", "<new_db_name>")\
    .option("tempDir", tempDir)\
    .save()

In [ ]:
# Write Data To an Existing Table
df1.write\
    .format("com.databricks.spark.sqldw")\
    .option("url", sqlDwUrl)\
    .option("forwardSparkAzureStorageCredentials", "true")\
    .option("dbTable", "<existing_table_name>")\
    .option("tempDir", tempDir)\
    .mode("overwrite")\
    .save()

In [ ]:
# Unmount ADLS Gen2 Storage Account Folder
dbutils.fs.unmount("/mnt/Gen2")

/mnt/Gen2 has been unmounted.
Out[63]: True

In [ ]:
# Query data from dedicated SQL Pool

df_query = spark.read \
   .format("com.databricks.spark.sqldw") \
   .option("url", sqlDwUrl) \
   .option("tempDir", tempDir) \
   .option("forwardSparkAzureStorageCredentials", "true") \
   .option("query", "select C_MKTSEGMENT, count(*) as Cnt from [dbo].[customer] group by C_MKTSEGMENT") \
   .load()

In [ ]:
df_query.count()

Out[21]: 5

In [ ]:
df_query.show()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3008728943279608> in <cell line: 1>()
----> 1 df_query.show()

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/sql/dataframe.py in show(self, n, truncate, vertical)
    609 
    610         if isinstance(truncate, bool) and truncate:
--> 611             print(self._jdf.showString(n, 20, vertical))
    612         else:
    613             try:

/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1319 
   1320         answer = self.gateway_

In [ ]:
display(df_query.limit(5))

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-3008728943279612> in <cell line: 1>()
----> 1 display(df_query.limit(5))

/databricks/python_shell/dbruntime/display.py in display(self, input, *args, **kwargs)
     81                     raise Exception('Triggers can only be set for streaming queries.')
     82 
---> 83                 self.add_custom_display_data("table", input._jdf)
     84 
     85         elif isinstance(input, list):

/databricks/python_shell/dbruntime/display.py in add_custom_display_data(self, data_type, data)
     34     def add_custom_display_data(self, data_type, data):
     35         custom_display_key = str(uuid.uuid4())
---> 36         return_code = self.entry_point.addCustomDisplayData(custom_display_key, data_type, data)
     37         ip_display({
     38             "application/vnd.databricks.v1+display": custom_display_key,

/databricks/s